In [1]:
import Pkg; Pkg.add("JuMP");  Pkg.add("GLPK");  Pkg.add("DelimitedFiles"); 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
using JuMP          # Modeling language for optimization
using GLPK          # Open-source solver for linear programming

In [3]:
# Dados do problema
n = 22  # número de operações
m = 10  # número de trabalhadores

10

In [4]:
production_times = [
    0.287857 0.830655 0.100000 0.248239 0.727981 0.655542 0.123721 1.341410 0.607163 0.650595 0.952860 0.516485 1.108985 0.390380 0.468933 1.001196 0.867240 0.500933 0.103134 0.034181 0.100000 0.865475 
    0.214674 0.677245 0.218670 0.334460 0.377926 0.098116 0.116527 0.373527 0.766943 0.487008 0.616509 0.981799 0.689655 1.183141 0.577251 0.656511 0.579161 0.253087 0.702422 0.577560 0.149161 1.524006 
    0.563387 0.373777 1.177244 0.407656 0.714569 0.766731 0.596989 1.386659 0.100000 0.364007 0.576406 0.248855 0.921155 0.100000 0.540836 0.496036 0.292919 0.890136 0.428335 0.616562 0.980137 1.443171 
    0.701306 0.372571 0.286429 0.776129 0.151218 0.430800 0.441205 0.592415 0.682784 0.963321 0.497589 0.215186 1.343249 0.560373 0.118498 0.416314 0.682767 1.117920 1.010403 0.441521 0.408943 1.073101 
    0.728880 0.339696 0.300568 0.461422 0.403869 0.577566 0.100000 0.290065 0.181337 0.100000 0.108598 0.100000 1.355450 0.952505 0.335884 0.100000 0.100000 0.057663 0.035723 0.106526 0.024241 1.451221 
    1.032563 0.461030 0.097871 0.139404 0.217762 0.172444 0.710332 0.921559 0.588515 0.860334 0.940454 0.519992 0.534624 0.973526 0.355370 0.321430 0.245714 0.752854 1.399322 0.225037 0.107761 0.707324 
    0.237756 0.346072 0.060057 0.195576 0.597975 0.487688 0.100000 0.235052 0.513761 0.321723 0.809187 0.463609 0.585087 0.310646 0.864026 0.737609 0.688433 0.669295 0.414836 0.509546 0.261491 1.424095 
    0.357533 0.330183 0.180000 0.126193 0.039703 0.100000 0.133739 0.460826 0.559455 0.478390 0.527090 0.100000 0.675302 0.638178 0.100000 0.254398 0.347360 0.415945 0.949802 0.100000 0.060807 0.913300 
    0.444417 0.100000 0.272252 0.522473 0.289680 0.365973 0.688991 0.691261 0.401780 0.407980 1.565759 0.100000 0.983368 0.870684 0.330318 0.346789 0.793784 0.720926 1.155663 0.062596 0.506984 0.071084 
    0.077435 0.100000 0.423076 0.429905 0.100000 0.784920 0.100000 0.632330 0.100000 0.645691 1.318457 0.215414 0.151356 1.517339 0.100000 0.661387 0.053080 0.522108 0.100000 1.481272 0.100000 0.111411 
]

10×22 Matrix{Float64}:
 0.287857  0.830655  0.1       0.248239  …  0.034181  0.1       0.865475
 0.214674  0.677245  0.21867   0.33446      0.57756   0.149161  1.52401
 0.563387  0.373777  1.17724   0.407656     0.616562  0.980137  1.44317
 0.701306  0.372571  0.286429  0.776129     0.441521  0.408943  1.0731
 0.72888   0.339696  0.300568  0.461422     0.106526  0.024241  1.45122
 1.03256   0.46103   0.097871  0.139404  …  0.225037  0.107761  0.707324
 0.237756  0.346072  0.060057  0.195576     0.509546  0.261491  1.4241
 0.357533  0.330183  0.18      0.126193     0.1       0.060807  0.9133
 0.444417  0.1       0.272252  0.522473     0.062596  0.506984  0.071084
 0.077435  0.1       0.423076  0.429905     1.48127   0.1       0.111411

In [5]:
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [6]:
# Variables
@variable(model, x[1:n, 1:m], Bin)  # Assignment of tasks to workers
@variable(model, b[1:n, 1:m], Bin)  # Start of a block for a worker
@variable(model, e[1:n, 1:m], Bin)  # End of a block for a worker
@variable(model, T >= 0)  # Maximum workload

T

In [7]:
@objective(model, Min, T)

T

In [8]:
for i in 1:n
    @constraint(model, sum(x[i, j] for j in 1:m) == 1)
end

In [9]:
# Contiguity constraints: Each worker has exactly one contiguous block
for j in 1:m
    # Exactly one start and one end per worker
    @constraint(model, sum(b[i, j] for i in 1:n) == 1)
    @constraint(model, sum(e[i, j] for i in 1:n) == 1)
    
    # Start position definition
    for i in 1:n
        # Handle boundary cases (i=1 and i=num_operations)
        prev_x = (i == 1) ? 0 : x[i-1, j]
        next_x = (i == n) ? 0 : x[i+1, j]
        
        # d[i,j] = 1 if x[i,j] = 1 and x[i-1,j] = 0
        @constraint(model, b[i, j] >= x[i, j] - prev_x)
        @constraint(model, b[i, j] <= x[i, j])
        @constraint(model, b[i, j] <= 1 - prev_x)
        
        # e[i,j] = 1 if x[i,j] = 1 and x[i+1,j] = 0
        @constraint(model, e[i, j] >= x[i, j] - next_x)
        @constraint(model, e[i, j] <= x[i, j])
        @constraint(model, e[i, j] <= 1 - next_x)
    end
end

In [10]:
#for j in 1:m
#    for i in 1:n
#        @constraint(model, b[i, j] + e[i, j] <= 1)
#    end
#end

In [11]:
# Workload constraints: T >= sum of production times for each worker
for j in 1:m
    @constraint(model, T >= sum(production_times[j, i] * x[i, j] for i in 1:n))
end

In [12]:
optimize!(model)

In [13]:
println("Maximum workload (T): ", objective_value(model))

# Print assignments
for j in 1:m
    assigned_tasks = [i for i in 1:n if value(x[i, j]) > 0.5]
    if !isempty(assigned_tasks)
        println("Worker $j: Tasks ", assigned_tasks, " | Workload: ", sum(production_times[j, i] for i in assigned_tasks))
    end
end

Maximum workload (T): 0.583828
Worker 1: Tasks [14] | Workload: 0.39038
Worker 2: Tasks [7, 8] | Workload: 0.490054
Worker 3: Tasks [9, 10] | Workload: 0.46400700000000006
Worker 4: Tasks [11] | Workload: 0.497589
Worker 5: Tasks [16, 17, 18, 19, 20, 21] | Workload: 0.42415300000000006
Worker 6: Tasks [15] | Workload: 0.35537
Worker 7: Tasks [1, 2] | Workload: 0.583828
Worker 8: Tasks [3, 4, 5, 6] | Workload: 0.44589599999999996
Worker 9: Tasks [22] | Workload: 0.071084
Worker 10: Tasks [12, 13] | Workload: 0.36677
